In [1]:
%run util.py

In [2]:
data_folder = '/home/jzh/Waze/'

In [3]:
import json

In [4]:
# Loading PL data
PL_dict = zload(data_folder + 'PL_dict.pkz') 

In [5]:
# PL_dict

In [6]:
mu_1 = PL_dict['PL_AM']  # normal PL for AM period
mu_2 = PL_dict['PL_MD']  # normal PL for MD period
mu_3 = PL_dict['PL_PM']  # normal PL for PM period
mu_4 = PL_dict['PL_NT']  # normal PL for NT period

mu_1 = mu_adjust(mu_1)  # normal PL
mu_01, mu1, mu_11, P1, G_11, H_11, U_11 = ChainGen_(mu_1)

mu_2 = mu_adjust(mu_2)  # normal PL
mu_02, mu2, mu_12, P2, G_12, H_12, U_12 = ChainGen_(mu_2)

mu_3 = mu_adjust(mu_3)  # normal PL
mu_03, mu3, mu_13, P3, G_13, H_13, U_13 = ChainGen_(mu_3)

mu_4 = mu_adjust(mu_4)  # normal PL
mu_04, mu4, mu_14, P4, G_14, H_14, U_14 = ChainGen_(mu_4)

<string>:486: RuntimeWarning: covariance is not positive-semidefinite.


In [7]:
beta = 0.001
G_list = [G_11, G_12, G_13, G_14]
H_list = [H_11, H_12, H_13, H_14]
U_list = [U_11, U_12, U_13, U_14]

eta_wc = {}
eta_Sanov = {}

# Get thresholds for Hoeffding's test corresponding to sample length n  
def etaWC(n):
    return HoeffdingRuleMarkovRobust_(beta, G_list, H_list, U_list, n)

In [8]:
# Loading test data

with open(data_folder + 'sample_path_jam_test_data_sorted_trans.json', 'r') as json_file:
    sample_path_jam_test_data_sorted_trans = json.load(json_file)

In [9]:
sample_path_jam_test_data_sorted_trans['0']

{u'alphaTrans': 86.0,
 u'startTime': 300.0054,
 u'uuid': [u'f48c594a-be63-38d4-87ff-cd316ecbf277',
  u'be49645b-78f7-371a-a2f1-6c1a62c625d8']}

In [10]:
i = 0  # index of windows
sam_gap = 120
win_size = 600
win_list = []
while i * sam_gap + win_size < 24 * 3600:
    win_list.append((i * sam_gap, i * sam_gap + win_size))
    i += 1

In [11]:
i

715

In [12]:
win_num = len(win_list)
jam_num = len(sample_path_jam_test_data_sorted_trans)

win_jam = {}
for i in range(win_num):
    key = str(i)
    value = []
    for j in range(jam_num):
        if ((sample_path_jam_test_data_sorted_trans[str(j)]['startTime'] >= win_list[i][0]) \
            and (sample_path_jam_test_data_sorted_trans[str(j)]['startTime'] < win_list[i][1])):
            value.append(str(j))
    win_jam[key] = value

In [13]:
win_jam['714'], win_list[714][0], win_num 

(['8139',
  '8140',
  '8141',
  '8142',
  '8143',
  '8144',
  '8145',
  '8146',
  '8147',
  '8148',
  '8149',
  '8150',
  '8151',
  '8152',
  '8153',
  '8154',
  '8155',
  '8156',
  '8157',
  '8158',
  '8159',
  '8160',
  '8161',
  '8162',
  '8163',
  '8164',
  '8165',
  '8166',
  '8167',
  '8168',
  '8169',
  '8170',
  '8171',
  '8172',
  '8173',
  '8174'],
 85680,
 715)

In [14]:
sample_path_jam_test_data_sorted_trans[win_jam['100'][0]]['startTime']

12300.1498

In [15]:
KL_list = []  # The list of the KL divergences; each is for a detection window
etaWC_list = []
for i in range(win_num):
    sample_path_jam = [sample_path_jam_test_data_sorted_trans[key]['alphaTrans'] for key in win_jam[str(i)]]
    n = len(sample_path_jam)
    if (n != 0):
        etaWC_list.append(etaWC(n))
        KL_1 = KL_est(sample_path_jam, mu_1)
        KL_2 = KL_est(sample_path_jam, mu_2)
        KL_3 = KL_est(sample_path_jam, mu_3)
        KL_4 = KL_est(sample_path_jam, mu_4)
        KL_list.append(min([KL_1, KL_2, KL_3, KL_4]))
    else:
        etaWC_list.append(1e-6)
        KL_list.append(0)

In [16]:
win_jam[str(100)], win_list[100][1]

(['429',
  '430',
  '431',
  '432',
  '433',
  '434',
  '435',
  '436',
  '437',
  '438',
  '439',
  '440',
  '441'],
 12600)

In [17]:
time_range = [win_list[i][0] / 60.0 for i in range(win_num)]
plot_points(time_range, KL_list, etaWC_list)
plt.ylabel('divergence')
plt.xlabel('time (min)')
pylab.ylim(-0.1, max(etaWC_list)+0.5)
pylab.xlim(0, max(time_range) + 10)
plt.savefig(data_folder + 'detec_results_WC_robust.pdf')
plt.show()

In [18]:
len(time_range), len(KL_list), len(etaWC_list), time_range[0:2]

(715, 715, 715, [0.0, 2.0])

In [19]:
potential_ano_list = []
for i in range(len(KL_list)):
    if KL_list[i] > etaWC_list[i]:
        for j in range(len(win_jam[str(i)])):
            potential_ano_list.append(win_jam[str(i)][j])

In [20]:
with open(data_folder + 'sample_path_jam_test_data_sorted.json', 'r') as json_file:
    sample_path_jam_test_data_sorted = json.load(json_file)

In [21]:
sample_path_jam_test_data_sorted['2917']

{u'alpha': 7.0,
 u'length': 601.0,
 u'numPts': 11.0,
 u'speed': 4.891666666666667,
 u'startTime': 52500.0489,
 u'uuid': u'c7ceccbd-5847-3213-8902-44f049362eb3'}

In [22]:
len(set(potential_ano_list)), set(potential_ano_list)

(979,
 {'1679',
  '1680',
  '1681',
  '1682',
  '1683',
  '1684',
  '1685',
  '1686',
  '1687',
  '1688',
  '1689',
  '1690',
  '1691',
  '1692',
  '1693',
  '1694',
  '1695',
  '1696',
  '1697',
  '1698',
  '1699',
  '1700',
  '1701',
  '1702',
  '1703',
  '1704',
  '1705',
  '1706',
  '1707',
  '1708',
  '1709',
  '1710',
  '1711',
  '1712',
  '1713',
  '1714',
  '1715',
  '1716',
  '1717',
  '1718',
  '1719',
  '1720',
  '1721',
  '1722',
  '1723',
  '1724',
  '1725',
  '1726',
  '1727',
  '1728',
  '1729',
  '1730',
  '1731',
  '1732',
  '1733',
  '1734',
  '1735',
  '1736',
  '1737',
  '1738',
  '1739',
  '1740',
  '1741',
  '1742',
  '1743',
  '1744',
  '1745',
  '1746',
  '1747',
  '1748',
  '1749',
  '1750',
  '1751',
  '1752',
  '1753',
  '1754',
  '1755',
  '1756',
  '1757',
  '1758',
  '1759',
  '1760',
  '1761',
  '1762',
  '1763',
  '1764',
  '1765',
  '1766',
  '1767',
  '1768',
  '1769',
  '1770',
  '1771',
  '1772',
  '1773',
  '1774',
  '1775',
  '1776',
  '1777',
  '1

In [23]:
len(sample_path_jam_test_data_sorted_trans)

8175

In [25]:
# Refine the detection results; if a jam's length is larger than 1 km,
# then the jam is "indeed" abnormal.
ano_list = []
for key in set(potential_ano_list):
    if (sample_path_jam_test_data_sorted[key]['length'] > 1000):
        ano_list.append(key)

In [26]:
len(ano_list), ano_list

(99,
 ['7482',
  '6721',
  '6528',
  '1942',
  '1941',
  '1795',
  '1708',
  '6654',
  '6753',
  '6596',
  '6385',
  '6382',
  '6503',
  '6502',
  '6506',
  '6436',
  '6808',
  '6806',
  '6804',
  '6613',
  '1995',
  '1994',
  '6755',
  '1692',
  '1755',
  '1699',
  '6415',
  '3412',
  '3413',
  '3419',
  '6470',
  '6797',
  '6850',
  '6853',
  '6790',
  '1717',
  '6581',
  '3398',
  '6536',
  '6363',
  '6361',
  '6693',
  '3444',
  '6409',
  '3441',
  '1964',
  '6697',
  '6864',
  '6865',
  '1686',
  '7457',
  '7476',
  '7396',
  '6568',
  '6562',
  '1929',
  '7368',
  '7366',
  '7438',
  '6820',
  '6829',
  '6674',
  '1729',
  '1722',
  '6676',
  '6548',
  '6684',
  '6524',
  '6526',
  '6481',
  '6350',
  '7445',
  '6557',
  '2032',
  '6639',
  '6718',
  '7388',
  '6644',
  '6392',
  '6396',
  '6708',
  '6692',
  '6457',
  '6514',
  '6426',
  '6422',
  '6423',
  '6731',
  '6603',
  '6608',
  '1981',
  '1730',
  '3375',
  '6460',
  '6494',
  '6493',
  '6492',
  '1762',
  '2006'])

In [27]:
for key in potential_ano_list:
    if (sample_path_jam_test_data_sorted[key]['startTime'] > 69900 and 
        sample_path_jam_test_data_sorted[key]['startTime'] < 69901):
        uuid = sample_path_jam_test_data_sorted[key]['uuid']
        print(points_dict_Mar16[uuid][0][2])

In [28]:
with open(data_folder + '/points_dict_Mar16_comp.json', 'r') as json_file:
    points_dict_Mar16_comp = json.load(json_file)

In [29]:
points_dict_Mar16_comp['50a5d058-2b16-3034-bd41-623f96b2829a']

[[[-71.126085, 42.330063],
  [-71.126085, 42.334441],
  [-71.123535, 42.330063],
  [-71.123535, 42.334441]],
 15]

In [30]:
('-71.128121', '42.359786', 'ACCIDENT', 68700.0, '2016-03-16 19:06:00.000')


('-71.128121', '42.359786', 'ACCIDENT', 68700.0, '2016-03-16 19:06:00.000')

In [31]:
jam1 = [42.359786, -71.128121]

In [32]:
jam2 = [42.363025, -71.13641] 

In [33]:
from geopy.distance import vincenty
vincenty(jam1, jam2).meters

771.8372664016924